In [1]:
import pickle
import pandas as pd
from datetime import datetime

In [3]:
with open('data/invoices_new.pkl', 'rb') as file:
    new = pickle.load(file)

print(new)

[{'id': '365371O', 'created_on': '2019-05-27', 'items': [{'item': {'id': 142446, 'name': 'ii_142446', 'unit_price': 182, 'type': 1}, 'quantity': 6}, {'item': {'id': 154019, 'name': 'ii_154019', 'unit_price': 176, 'type': 1}, 'quantity': 1}]}, {'id': 389528, 'created_on': '2019-03-19', 'items': [{'item': {'id': 172535, 'name': 'ii_172535', 'unit_price': 105, 'type': 0}, 'quantity': 4}, {'item': {'id': 175208, 'name': 'ii_175208', 'unit_price': 109, 'type': 0}, 'quantity': 7}, {'item': {'id': 139970, 'name': 'ii_139970', 'unit_price': 193, 'type': 2}, 'quantity': 4}, {'item': {'id': 156273, 'name': 'ii_156273', 'unit_price': 129, 'type': 2}, 'quantity': 6}, {'item': {'id': 178512, 'name': 'ii_178512', 'unit_price': 108, 'type': 1}, 'quantity': 7}, {'item': {'id': 191045, 'name': 'ii_191045', 'unit_price': 195, 'type': 1}, 'quantity': 8}, {'item': {'id': 189841, 'name': 'ii_189841', 'unit_price': 171, 'type': 1}, 'quantity': 4}]}, {'id': 341762, 'created_on': '2019-06-19', 'items': [{'ite

In [4]:
with open('data/expired_invoices.txt', 'r') as file:
    expired = file.read()

print(expired)

305869, 377307, 391273, 385290, 331902, 397723, 343695, 340601, 347510, 325156, 379387, 348894, 322229, 326649, 377960, 325063, 369378, 319405, 383681, 367288, 356552, 394428, 381476, 381457, 349879, 343254, 366751, 330931, 363263, 326452, 379687, 356532, 383235, 313012, 368913, 352442, 315960, 351096, 331193, 392657, 352391, 389528, 338547, 379961, 337140, 323231, 371205, 378746, 307175, 340299


In [6]:
class DataExtractor:
    def __init__(self, invoice_file, expired_file):
        self.invoice_file = invoice_file
        self.expired_file = expired_file
        self.invoices = None
        self.expired_ids = None

    def load_data(self):
        # Load the pickled invoices
        with open(self.invoice_file, 'rb') as f:
            self.invoices = pickle.load(f)
        
        # Load the expired invoice IDs
        with open(self.expired_file, 'r') as f:
            self.expired_ids = [int(line.strip()) for line in f.read().split(',')]
    
    def safe_int_conversion(self, value):
        try:
            return int(value)
        except (ValueError, TypeError):
            return None

    def safe_float_conversion(self, value):
        try:
            return float(value)
        except (ValueError, TypeError):
            return None
    
    def process_data(self):
        # Conversion table for invoice item types
        type_conversion = {0: 'Material', 1: 'Equipment', 2: 'Service', 3: 'Other'}
        
        processed_data = []

        for invoice in self.invoices:
            # Handle the id conversion properly based on its type
            if isinstance(invoice['id'], str):
                invoice_id = self.safe_int_conversion(invoice['id'].strip('O'))  # Convert '365371O' to 365371
            else:
                invoice_id = self.safe_int_conversion(invoice['id'])  # If it's already an int

            if invoice_id is None:
                continue  # Skip if invoice_id is invalid

            created_on = pd.to_datetime(invoice.get('created_on'), errors='coerce')
            is_expired = invoice_id in self.expired_ids

            # Ensure 'items' key exists
            if 'items' not in invoice:
                continue

            invoice_total = sum(self.safe_int_conversion(item['item'].get('unit_price')) * self.safe_int_conversion(item.get('quantity')) for item in invoice['items'] if self.safe_int_conversion(item['item'].get('unit_price')) is not None and self.safe_int_conversion(item.get('quantity')) is not None)
            
            for item in invoice['items']:
                item_id = self.safe_int_conversion(item['item'].get('id'))
                item_name = item['item'].get('name')
                item_type = type_conversion.get(self.safe_int_conversion(item['item'].get('type')), 'Unknown')
                unit_price = self.safe_int_conversion(item['item'].get('unit_price'))
                quantity = self.safe_int_conversion(item.get('quantity'))

                if None in (item_id, unit_price, quantity):
                    continue  # Skip if any critical field is invalid

                total_price = unit_price * quantity
                percentage_in_invoice = total_price / invoice_total if invoice_total else 0

                processed_data.append({
                    'invoice_id': invoice_id,
                    'created_on': created_on,
                    'invoiceitem_id': item_id,
                    'invoiceitem_name': item_name,
                    'type': item_type,
                    'unit_price': unit_price,
                    'total_price': total_price,
                    'percentage_in_invoice': percentage_in_invoice,
                    'is_expired': is_expired
                })

        return processed_data

    def to_dataframe(self):
        data = self.process_data()
        df = pd.DataFrame(data)
        
        # Ensure correct data types
        df['invoice_id'] = df['invoice_id'].astype(int)
        df['created_on'] = pd.to_datetime(df['created_on'])
        df['invoiceitem_id'] = df['invoiceitem_id'].astype(int)
        df['invoiceitem_name'] = df['invoiceitem_name'].astype(str)
        df['type'] = df['type'].astype(str)
        df['unit_price'] = df['unit_price'].astype(int)
        df['total_price'] = df['total_price'].astype(int)
        df['percentage_in_invoice'] = df['percentage_in_invoice'].astype(float)
        df['is_expired'] = df['is_expired'].astype(bool)
        
        # Sort by invoice_id and invoiceitem_id
        df = df.sort_values(by=['invoice_id', 'invoiceitem_id'])
        
        return df

# Example usage:
extractor = DataExtractor('data/invoices_new.pkl', 'data/expired_invoices.txt')
extractor.load_data()
df = extractor.to_dataframe()
print(df)


     invoice_id created_on  invoiceitem_id invoiceitem_name       type  \
291      301695 2019-04-26          103215        ii_103215      Other   
289      301695 2019-04-26          166227        ii_166227  Equipment   
292      301695 2019-04-26          171394        ii_171394      Other   
290      301695 2019-04-26          195625        ii_195625      Other   
129      304245 2019-03-17          121446        ii_121446    Service   
..          ...        ...             ...              ...        ...   
395      395841 2019-01-13          197505        ii_197505      Other   
86       397723 2019-05-11          117999        ii_117999  Equipment   
88       397723 2019-05-11          121772        ii_121772   Material   
85       397723 2019-05-11          181613        ii_181613   Material   
87       397723 2019-05-11          198198        ii_198198   Material   

     unit_price  total_price  percentage_in_invoice  is_expired  
291         135          945               0.

In [7]:
df.to_csv('resulting_dataframe.csv', index=False)
